In [9]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import pandas as pd
import urllib.parse
import konlpy
from konlpy.tag import Komoran
#database
import pymongo
import json
import datetime
from pymongo import MongoClient

try:
    driver= webdriver.Chrome('./chromedriver.exe')
except:
    driver= webdriver.Chrome('./chromedriver')
driver.implicitly_wait(3)
    
# 로그인 전용 화면
driver.get('https://nid.naver.com/nidlogin.login')

# id 입력

id_input_element = driver.find_element_by_css_selector("#id")

id_input_element.send_keys("jeungmin717")

# pw 입력

pw_input_element = driver.find_element_by_css_selector("#pw")

pw_input_element.send_keys("password")

# 네이버 로그인 완료 => 추후에 @로 만들어보기

submit_element = driver.find_element_by_css_selector(".btn_global")

submit_element.click()

In [10]:
keyword = input("찾으시려는 키워드를 입력하세요:")
page_number = input("찾으시려는 페이지 수를 입력하세요:")

#인코드한다 euc-kr 로 
b = keyword.encode('euc-kr')
keyword_euc = urllib.parse.quote(b)


#driver.get("http://cafe.naver.com/joonggonara.cafe?iframe_url=/ArticleList.nhn%3Fsearch.clubid=10050146%26search.boardtype=L%26viewType=pc")
#driver.get("http://cafe.naver.com/joonggonara/ArticleList.nhn?search.clubid=10050146")
#search.page="+page_number 마지막에 추가하여 페이지별이동가능.
driver.get("http://cafe.naver.com/joonggonara?iframe_url=/ArticleSearchList.nhn%3Fsearch.clubid=10050146%26search.media=0%26search.searchdate=all%26userDisplay=15%26search.option=0%26search.sortBy=date%26search.searchBy=1%26search.searchBlockYn=0%26search.query="+keyword_euc+"%26search.viewtype=title%26")

         
           #프레임 변환



찾으시려는 키워드를 입력하세요:아이폰 6S 로즈골드 32기가
찾으시려는 페이지 수를 입력하세요:1


In [7]:
res_list = []
article_urls= []


for i in range(int(page_number)):
    driver.get("http://cafe.naver.com/joonggonara?iframe_url=/ArticleSearchList.nhn%3Fsearch.clubid=10050146%26search.media=0%26search.searchdate=all%26userDisplay=15%26search.option=0%26search.sortBy=date%26search.searchBy=1%26search.searchBlockYn=0%26search.query="+keyword_euc+"%26search.viewtype=title%26search.page="+str(i+1))
    iframe_element = driver.find_element_by_css_selector("iframe#cafe_main")
    driver.switch_to_frame(iframe_element)
    # href 속성을 찾아 url을 리스트로 저장한다.
    article_list = driver.find_elements_by_css_selector('span.aaa > a.m-tcol-c')
    
    #article_urls = [ t.get_attribute('href') for t in article_list ]
    #이건 []로나온다.
    
    for t in article_list:
        article_urls.append(t.get_attribute('href'))
        
print(len(article_urls))





15


In [8]:
#시작부분 코드
import time
start_time = time.time() 
#------------------------
 
    
    
    # Beautifulsoup 활용
for article in article_urls:
    driver.get(article)
    # article도 switch_to_frame이 필수
    driver.switch_to_frame('cafe_main')
    soup = bs(driver.page_source, 'html.parser')
    # 제목,가격 검색
    title = soup.select('div.tit-box span.b')[0].get_text()
    try:
        #contents 초기화 이게맞나? 2015-05-15
        contents=[]
        #가격 검색
        price_div = soup.find('div', class_='prod_price')
        price = price_div.find('span').get_text()
        nick = soup.find('td', class_='p-nick' ).get_text()
        date = soup.find('td', class_='m-tcol-c date').get_text()
        #링크 가져오기
        link_url = soup.find('a',id='linkUrl').get_text()
        link_url = link_url[34:]
        
        #게시글을 띄어쓰기 단위로 합친다.
        #게시글내용
        #content_tags = soup.select('#tbody')[0].select('p')
        #content = ' '.join([ tags.get_text() for tags in content_tags ])
        
        
        
       
#         try:
#             temp = soup.find("div", id ="tbody").get_text()
#             print(temp)
#         except:
#             #NHNForm 의 경우
#             content_tags = soup.select('#tbody > div > div ')
#             for content_tag in content_tags:
#                 contents.append(content_tag.get_text())
#                 print(contents)
#                 contents = []
            
#2018-05-15 가작성 컨텐츠 가져오는 부분 완전하지않음 수정할 부분 : 특정 NHN 폼 못가져오고있음  + \xa0 같은 데이터가 낌 + 배열형태로 출력
        flag=0
        #NHNForm 의 경우
        content_tags = soup.select('#tbody > div > div ')
        #print(content_tags)
        print("-------------NHN 작성Form------------------")
        for content_tag in content_tags:
            if(flag==1): #수정시작
                #contents="".join(content_tag.get_text().replace(u'\xa0', u' '))
                contents.append(content_tag.get_text().replace(u'\xa0', u' '))    
                #contents = ' '.join(content_tag.get_text())
                #print(contents)
            else:
                contents = []
                flag=1
        
        
        
        # <p> 인경우 (원래)
        content_tags = soup.select('#tbody > p')
        print("---------------<p>태그에서 가져옴-----------")
        for content_tag in content_tags:
            contents.append(content_tag.get_text().replace(u'\xa0', u' '))
            #contents.append(content_tag.get_text())
            
        
        #마지막으로 컨텐트 출력
        contents="".join(contents)
        
        #형태소 분석.
        komoran = Komoran()
        print(komoran.nouns(contents))
    
    
    
    
    
        if soup.find("em", class_='sp_end sale_complete'):
            print("판매완료상품")
            status='sold_out'
        else:
            print("판매중인상품")
            status='selling'
        
        #dict형태로 만들어 결과 post에 저장
        
        post = {'id':link_url,'title' : title,'nick' : nick, 'date' : date ,'price' : price, 'content' : contents, 'status': status,
        'date': datetime.datetime.utcnow()}
        
        #post 를 joongonara 몽고db에 저장.
        connection = pymongo.MongoClient("localhost", 27017)
        db = connection.joongonara
        collection  = db.joongonara
        collection.insert_one(post)
        
        
        
        #res_list.append({'id':link_url,'title' : title,'nick' : nick, 'date' : date ,'price' : price, 'content' : contents, 'status': status })
        
        print(title)
        print(price)
        print(nick)
        print(date)
        print(link_url)
        print("-------------------------------------------")
    except Exception as ex: # 에러 종류
        print('에러가 발생 했습니다', ex) # ex는 발생한 에러의 이름을 받아오는 변수

#time.sleep 작업도 필요하다.
#결과 데이터프레임화
cafe_df = pd.DataFrame(res_list)
#csv파일로 추출
cafe_df.to_csv('cafe_crawling_new.csv', index=False, encoding='UTF-8')
print("성공")
    
    

 
#----------------------------
#종료부분 코드
print("start_time", start_time) #출력해보면, 시간형식이 사람이 읽기 힘든 일련번호형식입니다.
print("--- %s seconds ---" %(time.time() - start_time))



-------------NHN 작성Form------------------
---------------<p>태그에서 가져옴-----------
['해밀턴', '구형', '카키', '필드', '판매', '브이', '사이즈', '사파이어', '크리스탈', '상처', '양호', '상태', '시계', '정상', '작동', '오버', '홀', '필요', '구성', '품', '사진', '본체', '판매', '가격', '만', '연락']
판매중인상품
해밀턴 구형 카키필드 38mm
230,000원
dirtyangel2(dirt****)
2018.06.04. 15:18
481747313
-------------------------------------------
-------------NHN 작성Form------------------
---------------<p>태그에서 가져옴-----------
['해밀턴', '카키', '필드', '쿼츠', '제품', '월', '일', '신세계', '면세점', '구매', '번', '착용', '제품', '선물', '추천', '직거래', '택배']
판매완료상품
해밀턴 카키필드 쿼츠 38mm 새제품
250,000원
내장탕(drco****)
2018.06.02. 11:24
481141720
-------------------------------------------
-------------NHN 작성Form------------------
---------------<p>태그에서 가져옴-----------
[]
판매중인상품
해밀턴 카키필드 오토 38mm 42만원 
420,000원
ergtt(bbon****)
2018.05.31. 15:43
480636402
-------------------------------------------
-------------NHN 작성Form------------------
---------------<p>태그에서 가져옴-----------
['해밀턴', '카키', '필드', '쿼츠', '제품', '월',